In [1]:
# Import required libraries and dependencies
import pandas as pd                                  # For data manipulation and handling  
import hvplot.pandas                                 # For interactive plotting using hvPlot 
from sklearn.cluster import KMeans                   # Import K-Means for clustering  
from sklearn.decomposition import PCA                # Import PCA for dimensionality reduction  
from sklearn.preprocessing import StandardScaler     # Import StandardScaler for feature scaling  

# Load crypto data into a DataFrame.

In [2]:
# Load the data into a Pandas DataFrame. Set coin_id as the index column
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384


In [3]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(  # Create a line plot using the hvplot library on the df_market_data DataFrame
    width=800,          # Set the plot width to 800 pixels
    height=400,         # Set the plot height to 400 pixels
    rot=90              # Rotate the x-axis labels by 90 degrees for better readability
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

# Defining a function to calculate k and inertia and then to plot a line graph using hvplot.

In [5]:
 # Define a function that takes a DataFrame as input.

def k_calculate_plot(dataframe):

    # Create a list with the number of k-values from 1 to 11
    k = list(range(1,11))

    # Create an empty list to store the inertia values
    inertia = []

# Loop through possible values of k (number of clusters)
    for i in k:

        # Create a KMeans clustering model with 'i' clusters and a fixed random state for reproducibility
        model = KMeans(n_clusters=i ,  random_state=1)

        # Fit the KMeans model to the given dataset
        model.fit(dataframe)

        # Append the inertia (sum of squared distances to the nearest cluster center) to the list
        inertia.append(model.inertia_)

    # Create a dictionary to store k-values and corresponding inertia values
    elbow_dict = { "k" : k ,
                "inertia" : inertia}

    # Convert the dictionary into a DataFrame for plotting
    elbow_data = pd.DataFrame(elbow_dict)

# Create an Elbow curve plot using hvplot to visualize the optimal number of clusters
    elbow_plot = elbow_data.hvplot.line(x="k",          # Set the x-axis as the number of clusters (k)
                        y="inertia",                    # Set the y-axis as the inertia (within-cluster sum of squares)
                        title=f"Elbow Curve",           # Title of the plot
                        xticks=k                        # Ensure x-axis ticks correspond to k values
                        )
    return elbow_plot


# Defining a function to plot a scatter plot for the scaled dataframes..

In [6]:
# Define a function that takes a DataFrame and two column names (x and y) as input.
def scatter_plot(dataframe,x,y):

    # Create and return a scatter plot using hvPlot.
    return dataframe.hvplot.scatter(
        x=x,                # Set the x-axis using the provided column name
        y=y,                # Set the y-axis using the provided column name
        by="Clusters",      # Color the points based on their assigned cluster labels (from K-Means clustering).
        hover_cols=["coin_id"]   # Add a hover feature displaying the cryptocurrency's ID when hovering over points.
    )


---

# Defining a function to cluster cryptocurrencies for the best value of k on the scaled DataFrame

In [7]:
def crypto_cluster(data, k):

   # Initialize the K-Means model using the best value for k
    k_model =  KMeans(n_clusters=k, random_state=1)     

    # Fit the K-Means model using the scaled DataFrame
    k_model.fit(data)

    # Predict the clusters to group the cryptocurrencies using the scaled DataFrame
    kmeans_prediction =  k_model.predict(data)

    # Create a copy of the scaled DataFrame
    df_market_data_scaled_prediction = data.copy()

    # Add a new column to the copy of the scaled DataFrame with the predicted clusters
    df_market_data_scaled_prediction["Clusters"] = kmeans_prediction
    
    return df_market_data_scaled_prediction

# Prepare the Data

##### Normalize the numerical columns to ensure that all features have a mean of 0 and a standard deviation of 1. This is important because some machine learning models (like k-means, PCA, and regression) perform better when features are on a similar scale.

In [8]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()                               # Create an instance of StandardScaler to standardize the data.

# Apply scaling (standardization) to the selected columns.
df_market_data_scaled = scaler.fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']])

# Display the data
df_market_data_scaled[:5]

array([[ 0.50852937,  0.49319307,  0.77220043,  0.23545963, -0.0674951 ,
        -0.35595348, -0.25163688],
       [ 0.18544589,  0.93444504,  0.55869212, -0.05434093, -0.27348273,
        -0.11575947, -0.19935211],
       [ 0.02177396, -0.70633685, -0.02168042, -0.06103015,  0.00800452,
        -0.55024692, -0.28206051],
       [-0.04076438, -0.81092807,  0.24945797, -0.05038797, -0.37316402,
        -0.45825882, -0.29554614],
       [ 1.19303608,  2.00095907,  1.76061001,  0.54584206, -0.29120287,
        -0.49984776, -0.27031695]])

In [9]:
# Create a DataFrame with the scaled data

df_market_data_scaled = pd.DataFrame(df_market_data_scaled, columns=['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'])

# Rounding data upto 6 decimals
df_market_data_scaled =  df_market_data_scaled.round(6)

# Copy the crypto names from the original DataFrame
df_market_data_scaled["coin_id"] = df_market_data.index

# Set the coin_id column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display the scaled DataFrame
df_market_data_scaled.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Scaled DataFrame.

In [10]:
# Call k_calculate_plot function to compute k and inertia and for plotting the elbow graph.
elbow_plt = k_calculate_plot(df_market_data_scaled)
elbow_plt 

c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is k

:Curve   [k]   (inertia)

#### Answer the following question: 

**Question:** What is the best value for `k`?

Answer: The best value of k is 4 because at k=4, there is a drastic decrease in inertia.Beyond this point(k=4), the drop in inertia slows down.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [11]:
# Call crypto_cluster function to cluster the cryptocurrencies for the best value of k on the the scaled dataframe
original_scaled_data = crypto_cluster(df_market_data_scaled, 4) # here 4 is the best value of k.
original_scaled_data.head()


c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Clusters
coin_id,,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637,2
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352,2
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061,0
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546,0
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317,2


In [12]:
#Call the functon scatter_plot to create scatter plot of the scaled_original_dataframe
original_scatter_plot = scatter_plot(original_scaled_data, "price_change_percentage_24h", "price_change_percentage_7d")
original_scatter_plot

:NdOverlay   [Clusters]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [13]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [14]:
# Use the PCA model with `fit_transform` to reduce the original scaled DataFrame
# down to three principal components.
# This reduces the dimensionality of the dataframe to 3
df_market_data_scaled_pca = pca.fit_transform(df_market_data_scaled) 

# View the scaled PCA data
df_market_data_scaled_pca[:3]

array([[-0.60066627,  0.84276034,  0.46159441],
       [-0.45826018,  0.45846572,  0.9528769 ],
       [-0.43307004, -0.16812548, -0.64175221]])

In [15]:
# Retrieve the explained variance to determine how much information can be attributed to each principal component.
pca.explained_variance_ratio_

array([0.37198559, 0.34700813, 0.17603793])

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:**  About 89% of the total variance is condensed into the 3 PCA variables

In [16]:
# Create a new DataFrame with the PCA data.
df_market_data_scaled_pca = pd.DataFrame(df_market_data_scaled_pca,
                                         columns=["PC1","PC2","PC3"])

# Copy the crypto names from the original scaled DataFrame
df_market_data_scaled_pca["coin_id"] = df_market_data_scaled.index

# Set the coin_id column as index
df_market_data_scaled_pca = df_market_data_scaled_pca.set_index("coin_id")
df_market_data_scaled_pca = df_market_data_scaled_pca.round(6)

# Display the scaled PCA DataFrame
df_market_data_scaled_pca.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-0.600666,0.842760,0.461594
ethereum,-0.458260,0.458466,0.952877
tether,-0.433070,-0.168125,-0.641752
ripple,-0.471835,-0.222660,-0.479053
bitcoin-cash,-1.157799,2.041210,1.859715


---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [17]:
# Call the function k_calculate_plot to calculate k and corresponding inertia and then plot the elbow graph
pca_elbow_plt = k_calculate_plot(df_market_data_scaled_pca)
pca_elbow_plt

c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is k

:Curve   [k]   (inertia)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value of k seems to be k = 4 as there is a sudden drop in inertia at k=4.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** No, the best value of k  doesn't differ from the value of k found using original data. 

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [18]:
# Call the function crypto_cluster to create clustered cryptocurrencies dataframe for the best value of k
pca_data = crypto_cluster(df_market_data_scaled_pca, 4)
pca_data.head()

c:\Users\rince\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1429: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


,PC1,PC2,PC3,Clusters
coin_id,,,,
bitcoin,-0.600666,0.842760,0.461594,2
ethereum,-0.458260,0.458466,0.952877,2
tether,-0.433070,-0.168125,-0.641752,0
ripple,-0.471835,-0.222660,-0.479053,0
bitcoin-cash,-1.157799,2.041210,1.859715,2


In [19]:
#Call the functon scatter_plot to create scatter plot of the scaled PCA dataframe
pca_scatter_plot = scatter_plot(pca_data, "PC1", "PC2")
pca_scatter_plot

:NdOverlay   [Clusters]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [22]:
# Composite plot to contrast the Elbow curves
# Overlay two line plots
overlay_elbow_plots = elbow_plt + \
               pca_elbow_plt

overlay_elbow_plots

:Layout
   .Curve.I  :Curve   [k]   (inertia)
   .Curve.II :Curve   [k]   (inertia)

In [23]:
# Composite plot to contrast the clusters
overlay_scatter_plots = original_scatter_plot +\
    pca_scatter_plot
overlay_scatter_plots

:Layout
   .NdOverlay.I  :NdOverlay   [Clusters]
      :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)
   .NdOverlay.II :NdOverlay   [Clusters]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** On using fewer features to cluster the data, the cluster tend to appear more compressed and the boundaries seems to blur, resulting less distinct groups.